In [1]:
from project.data.data_loaders import get_clean_train_data
from sklearn.model_selection import GroupKFold
from project.experiments.theoviel_reproduction.reproduced_model import TrainDataset
from torch.utils.data import DataLoader
from project.experiments.theoviel_reproduction.reproduced_model import CFG, DebertaCustomModel, get_optimizer, get_scheduler, train_loop, val_loop, get_f1_score

data = get_clean_train_data()
train_indices, test_indices = list(GroupKFold(n_splits=5).split(data['pn_history'], data['location'], data['pn_num']))[0]
train_data = data.iloc[train_indices]
valid_data = data.iloc[test_indices]

assert not train_data.pn_num.isin(valid_data.pn_num).any()
print(f'Train data: {train_data.pn_num.nunique()}, valid data: {valid_data.pn_num.nunique()}')
train_dataset = TrainDataset(train_data)
valid_dataset = TrainDataset(valid_data)

train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

Using cache dir: /Users/evanvogelbaum/.cache, please set environment variable TRANSFORMERS_CACHE to override
Train data: 800, valid data: 200


In [2]:
model = DebertaCustomModel(f'/Users/evanvogelbaum/nbme_comp/src/project/data/deberta_training_saves/epoch2_state_dict.pt')


Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaModel: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading model from /Users/evanvogelbaum/nbme_comp/src/project/data/deberta_training_saves/epoch2_state_dict.pt


In [3]:
text = train_data['pn_history'].iloc[1]
feature = train_data['feature_text'].iloc[1]
model.inference(text, feature, 0)

[(667, 692)]

In [4]:
import torch
optimizer = get_optimizer(model)
scheduler = get_scheduler(optimizer, int(len(train_data) / CFG.batch_size * CFG.epochs))
train_loop(model, train_loader, optimizer, scheduler, torch.device('cpu'))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Using cache dir: /Users/evanvogelbaum/.cache, please set environment variable TRANSFORMERS_CACHE to override
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible


  0%|          | 0/953 [00:00<?, ?it/s]/Users/evanvogelbaum/opt/anaconda3/envs/PersonalCoding/lib/python3.10/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


tensor([[-8.8105e+00],
        [ 1.6819e-04],
        [ 3.0549e-04],
        ...,
        [-2.6014e+00],
        [-2.4921e+00],
        [-2.1512e+00]], grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) <class 'torch.Tensor'>


  0%|          | 0/953 [00:26<?, ?it/s]


TypeError: _softmax_backward_data(): argument 'input_dtype' (position 4) must be torch.dtype, not Tensor